In [ ]:
!git add .
!git commit -m "Updated notebook with new results"
!git push origin main


In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import ollama

# === Load everything once ===
print("🚀 Initializing models and data...")

# Load FAISS index
index = faiss.read_index("fitgen_vector_db/fitgen_index.faiss")

# Load chunked text
with open("fitgen_vector_db/fitgen_chunks.txt", "r", encoding="utf-8") as f:
    chunk_lines = [line.strip() for line in f.read().split("\n\n") if line.strip()]

# Load SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)
if device == "cuda":
    model = model.half()  # Use float16 on GPU for faster inference

# === Utility: truncate each chunk ===
def truncate(text, max_tokens=100):
    return ' '.join(text.split()[:max_tokens])

# === Main Loop ===
while True:
    query = input("\n🔍 Ask your question (or 'exit'): ").strip()
    if query.lower() == 'exit':
        break

    # Encode query
    query_vec = np.array(model.encode([query], convert_to_numpy=True), dtype='float32')

    # Search in FAISS
    k = 3
    _, I = index.search(query_vec, k)

    # Get and truncate top chunks
    top_chunks = [truncate(chunk_lines[i], 100) for i in I[0]]

    print("\n📚 Top Retrieved Chunks:")
    for i, chunk in enumerate(top_chunks, 1):
        print(f"\n--- Chunk {i} ---\n{chunk}")

    # Build prompt for RAG
    context = "\n\n".join(top_chunks)
    prompt = f"""Use the following context to answer the user's question concisely and clearly.

Context:
{context}

Question: {query}
Answer:"""

    print("\n🧠 Generating answer with LLaMA 3 (Ollama)...")

    try:
        response = ollama.chat(
            model='llama3',  # Faster quantized model
            messages=[
                {'role': 'user', 'content': prompt}
            ],
            options={
                'temperature': 0.5,
                'num_predict': 150
            }
        )
        answer = response['message']['content'].strip()
        print("\n💬 Answer:\n", answer)

    except Exception as e:
        print(f"\n❌ Ollama error: {e}")
        print("Ensure Ollama is running and llama3:8b-q4 is pulled with 'ollama run llama3:8b-q4'.")

print("👋 Session ended.")



🚀 Initializing models and data...



🔍 Ask your question (or 'exit'):  i m 24 yers old 183 cm and weight 90 kg give a starter workout



📚 Top Retrieved Chunks:

--- Chunk 1 ---
### ELITE SWIMMING WORKOUT_full_text.txt Expands the size of your hands, to work on your pull. Get a size that works for your hand, I wouldn’t recommend going overboard with the monster sized ones. Buoy: To put in between your legs and to stop kicking, and for rotations. Bread and butter of swim workouts. Chutes: Strap it to your waist and significantly increase the resistance you feel in the water. Must have for sprinters. Med Balls: Like a basketball, but heavy. Provides just the right medicine you need to get your muscles in shape. *All the of the workouts are designed

--- Chunk 2 ---
### MASTERING SWIMMING_full_text.txt raises your heart rate into an aerobic zone (60 to 70 percent of MHR). After the warm-up, move through the workout from the largest muscle groups (abdominals) to the smaller, more specific ones (arms). Finally, scheduling 10 minutes to relax and stretch your muscles at the conclusion of your session will reduce any soreness

KeyboardInterrupt: Interrupted by user


🔍 Ask your question (or 'exit'):  exit


In [5]:
import os 
os.environ["TOGETHER_API_KEY"] = "fa1509060a01b6917baaa50c6a09c4f63c385044e57153b6402599517b6d135d"


In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from together import Together
import os

# === 🔑 API Key (make sure it's set in environment) ===
# export TOGETHER_API_KEY=your_key_here
client = Together()  # Reads from os.environ["TOGETHER_API_KEY"]
MODEL_NAME = "deepseek-ai/DeepSeek-V3"
K = 10
MAX_TOKENS = 150
CHUNK_TRUNCATE_TOKENS = 100

# === 🚀 Load FAISS index and chunks ===
print("🔄 Loading FAISS index and chunks...")
index = faiss.read_index("fitgen_vector_db/fitgen_index.faiss")
with open("fitgen_vector_db/fitgen_chunks.txt", "r", encoding="utf-8") as f:
    chunk_lines = [line.strip() for line in f.read().split("\n\n") if line.strip()]

# === 🔠 Load SentenceTransformer for embedding ===
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)
if device == "cuda":
    model = model.half()

def truncate(text, max_tokens=100):
    return ' '.join(text.split()[:max_tokens])

# === 🧠 RAG Loop ===
while True:
    query = input("\n🔍 Ask your question (or 'exit'): ").strip()
    if query.lower() == "exit":
        break

    # Encode and search
    query_vec = np.array(model.encode([query], convert_to_numpy=True), dtype='float32')
    _, I = index.search(query_vec, k=K)
    top_chunks = [truncate(chunk_lines[i], CHUNK_TRUNCATE_TOKENS) for i in I[0]]

    # Build prompt
    context = "\n\n".join(top_chunks)
    prompt = f"""You are a helpful assistant.

Use the following context to answer the user's question concisely.

Context:
{context}

Question: {query}
Answer:"""

    print("\n🧠 Generating with DeepSeek-V3 from Together.ai...")

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=MAX_TOKENS,
            temperature=0.7
        )
        answer = response.choices[0].message.content.strip()
        print("\n💬 Answer:\n", answer)

    except Exception as e:
        print(f"\n❌ Error from Together API: {e}")

print("👋 Done.")


🔄 Loading FAISS index and chunks...



🔍 Ask your question (or 'exit'):  how to recover faster



🧠 Generating with DeepSeek-V3 from Together.ai...

💬 Answer:
 To recover faster after swimming sessions, focus on these key strategies from the provided context:

1. **Hydration & Fueling**: Sip water between sessions to avoid dehydration and prevent complete fuel depletion, which slows recovery.  
2. **Balanced Training**: Mix hard work with adequate recovery to avoid overtraining and maintain energy levels.  
3. **Hydrotherapy**: Use cold-water immersion, contrast baths (alternating hot/cold water), or low-intensity swimming to reduce muscle fatigue.  
4. **Rest**: Prioritize sleep, including short naps (30–60 minutes), if possible.  
5. **Pacing**: Avoid rushing anaerobic training—progress gradually ("make haste slowly") to prevent burnout.  
6. **Relaxed
